# ASP solving with clingo in Python

This is a small example to show how you can do Answer Set Programming with [clingo](https://potassco.org/clingo/) (part of the Potsdam Answer Set Solving Collection; or *Potassco*, for short) in Python.

First, install clingo, e.g., using:
`conda install -c potassco clingo`

To use clingo in Python, import the clingo package.

In [1]:
import clingo

You can then write an answer set program as a string.

For an explanation of the syntax of answer set programs (that clingo uses) and for examples, see Potassco's [Getting Started page](https://potassco.org/doc/start/) and their [guide](https://github.com/potassco/guide/releases/tag/v2.2.0).

Let's take the following simple example.

In [2]:
asp_program = """#const k=3.
number(1..k).
left(X) :- not right(X), number(X).
right(X) :- not left(X), number(X).
:- right(2).""";

We then create a clingo Control object, load the answer set program, and do the grounding (compiling away variables).

In [3]:
control = clingo.Control()
control.add("base", [], asp_program)
control.ground([("base", [])])

Before we ask clingo to find models (answer sets) for our program, we define a function `on_model` that will be called for each model.

For this example, we let this function extract and print some information about literals in the answer set that use the predicates `left/1` and `right/1`.

In [4]:
def on_model(model):
    print("ANSWER SET:")
    for atom in model.symbols(atoms=True):
        if atom.name == "right":
            print("Right: {}".format(
                atom.arguments[0].number
            ))
        elif atom.name == "left":
            print("Left: {}".format(
                atom.arguments[0].number
            ))

We then ask clingo to find a two answer sets for our program, and to call `on_model` on these when they are found.

In [5]:
control.configuration.solve.models = 2; # use 0 if you want to find all models
answer = control.solve(on_model=on_model)

ANSWER SET:
Right: 1
Right: 3
Left: 2
ANSWER SET:
Right: 1
Left: 2
Left: 3


We can check whether `solve()` found a model or not:

In [6]:
if answer.satisfiable == True:
    print("Found at least one model!")
else:
    print("Did not find a model!")

Found at least one model!


For more information, see the [clingo Python API](https://potassco.org/clingo/python-api/5.4/).